In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# [초기화 시작]

cred = credentials.Certificate('kiosk-firestore-jnsy-bcfe4-firebase-adminsdk-269q4-4a795774bb.json')
firebase_admin.initialize_app(cred)

# 8가지 감정 키워드
emotion_keyword = ["anger","contempt","disgust","fear","happiness","neutral","sadness","surprise"]

emotion_df = pd.DataFrame({ 'emotion_keyword':emotion_keyword })
emotion_vec = pd.get_dummies(emotion_df).values
#print(emotion_vec)

emotion_keyword_to_vec = {}
for index,keyword in enumerate(emotion_keyword):
    emotion_keyword_to_vec[keyword] = ''.join(list(map(str,emotion_vec[index].tolist())))
#emotion_keyword_to_vec: dict
print(emotion_keyword_to_vec)

# [초기화 끝]

{'anger': '10000000', 'contempt': '01000000', 'disgust': '00100000', 'fear': '00010000', 'happiness': '00001000', 'neutral': '00000100', 'sadness': '00000010', 'surprise': '00000001'}


In [ ]:
docs = firestore.client().collection(u'order').stream() #get 사용하지 말란다

doc_list = []
for doc in docs:
    #print(doc.get('emotion'))
    #print(type(doc))
    doc_list_row = []
    
    # 메뉴 - 레이블
    doc_list_row.append(doc.get('orderToString'))
 
    # 날씨
    weather_key_list = []
    for key in doc.get('weather').keys():
        #weather_key_list.append(doc.get('weather').get(key))
        doc_list_row.append(doc.get('weather').get(key))
    
    # 감정
    emotion_key_list = ''
    for key in doc.get('emotion').keys():
        #print(key)
        emotion_key_list += emotion_keyword_to_vec.get(key)
    doc_list_row.append(emotion_key_list)
    
    doc_list.append(doc_list_row)
    
doc_df = pd.DataFrame(doc_list, columns=['item_name','w_humidity','w_temp','w_speed','emotion'])

#레이블은 레이블 인코딩
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
items = doc_df['item_name'].values.tolist()
encoder.fit(items)
#labels = encoder.transform(items)
doc_df['item_name'] = pd.Series(encoder.transform(items))

print('인코딩 클래스:', encoder.classes_)
print('디코딩 원본 값:', encoder.inverse_transform([0,1]))
doc_df

In [ ]:
doc_df['item_name'] = pd.Series(encoder.transform(items))
doc_df['item_name']

In [ ]:
print(label_list)
print(doc_df['item_name'])
label_df

In [ ]:
#학습

dt_clf = DecisionTreeClassifier()
X_train,X_test,y_train,y_test = train_test_split(doc_df[['w_humidity','w_temp','w_speed','emotion']],
                                                doc_df[['item_name']], test_size=0.2)
dt_clf.fit(X_train,y_train)

pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('결정 트리 예측 정확도: {0:.4f}'.format(accuracy))

In [ ]:
# 람다 test
# 람다에 매개: humidity, temp, speed, emotion 보내서 정제해야 겠다
# emotion 변환은 DB에 저장해서 가져와야겠는걸..? 아닌가 람다 초기화에 이용하면 되나? 전역
#  event[''] - humidity, temp, speed, emotion
label_df = pd.DataFrame({ 'w_humidity':[80.0],#event['humidity']],
                         'w_temp':[300.0],#event['temp']],
                         'w_speed':[1.00],#event['speed']],
                         'emotion':['0100000000000001']})#event['emotion']] })
print('param', label_df)
pred2 = dt_clf.predict(label_df)
print('predict: ', pred2)